In [ ]:
!pip install git+https://github.com/speechbrain/speechbrain.git@develop

Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/speechbrain/speechbrain.git (to revision develop) to /tmp/pip-req-build-wxnzqepj
  Running command git clone --filter=blob:none --quiet https://github.com/speechbrain/speechbrain.git /tmp/pip-req-build-wxnzqepj
  Resolved https://github.com/speechbrain/speechbrain.git to commit e670108b1069d645791891503e4f669211be4560
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [ ]:
!pip install audiofile

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [ ]:
import torchaudio
import os
import librosa
import torch
import pickle as pkl
import audiofile
from soundfile import LibsndfileError
from speechbrain.inference.classifiers import EncoderClassifier
language_id = EncoderClassifier.from_hparams(source="speechbrain/lang-id-voxlingua107-ecapa", savedir="tmp")

/home/marcel/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
librosa.load(file, sr=16000)[0]

/usr/lib/python3.10/inspect.py:869: UserWarning: Module 'speechbrain.pretrained' was deprecated, redirecting to 'speechbrain.inference'. Please update your script. This is a change from SpeechBrain 1.0. See: https://github.com/speechbrain/speechbrain/releases/tag/v1.0.0
  if ismodule(module) and hasattr(module, '__file__'):


NameError: name 'file' is not defined

In [ ]:
path = os.getcwd() # to be run on same directory as audio files
files = os.listdir(path)
supported_audio_formats = ["wav","mp3"]
files = [file for file in files if file[-3:] in supported_audio_formats]

In [ ]:
path

'/home/marcel/Descargas/uploads'

In [ ]:
file_ends = set()
for file in files:
    file_ends.add(file[-3:])
file_ends

{'mp3', 'wav'}

In [ ]:
signal, sampling_rate = audiofile.read("municipalityofvalencia_JustinaPerezCantos_m70531_a87275_audio.wav")
signal

LibsndfileError: Error opening '/home/marcel/Descargas/uploads/municipalityofvalencia_JustinaPerezCantos_m70531_a87275_audio.wav': Supported file format but file is malformed.

In [ ]:
predictions = []
corrupted_files = []
predictions_file = "predictions_list.pkl"
i = 0
for file in files[1:]:
    #print(file)
    i+=1
    try:
        signal, sr = audiofile.read(file)
    except LibsndfileError as e:
        print("Corrupt file",file)
        corrupted_files.append(file)
        continue
    prediction = language_id.classify_batch(torch.tensor(signal))
    predictions.append(prediction[3][0].split(":")[1].strip())
    if i % 100 == 0:
        print(f"Iteration {i} completed reaching update...")
        with open(path+"/"+predictions_file,"wb") as f:
            pkl.dump(predictions,f)

with open(path+"/"+predictions_file,"wb") as f:
    pkl.dump(predictions,f)

NameError: name 'files' is not defined